In [2]:
import numpy as np

def fdr_correction(p_values, alpha=0.05):
    p_values = np.array(p_values)
    m = len(p_values)  # Total number of tests
    sorted_indices = np.argsort(p_values)  # Indices that would sort the p-values
    sorted_p_values = p_values[sorted_indices]  # Sorted p-values

    # Compute adjusted p-values
    adjusted_p_values = np.empty(m)
    cumulative_min = 1.0  # Start with the largest p-value in reverse order
    for i in range(m - 1, -1, -1):
        rank = i + 1  # Rank in sorted list (1-based)
        adjusted_p_value = (m / rank) * sorted_p_values[i]
        cumulative_min = min(cumulative_min, adjusted_p_value)  # Ensure monotonicity
        adjusted_p_values[i] = cumulative_min

    # Return p-values to their original order
    adjusted_p_values = np.clip(adjusted_p_values, 0, 1)  # Ensure all adjusted p-values are <= 1
    adjusted_p_values_original_order = np.empty(m)
    adjusted_p_values_original_order[sorted_indices] = adjusted_p_values

    return adjusted_p_values_original_order

In [4]:
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

# Generate example data
np.random.seed(42)
n_obs = 100  # Number of observations
n_preds = 10  # Number of predictors

# Generate random predictors and response variable
X = np.random.rand(n_obs, n_preds)  # Predictor variables
y = 3 * X[:, 0] + 2 * X[:, 1] + np.random.randn(n_obs)  # Response variable with signal in X[:, 0] and X[:, 1]

# Add an intercept to X
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Extract p-values for all predictors (excluding the intercept)
p_values = model.pvalues[1:]  # Skip the intercept

# Apply Benjamini-Hochberg FDR correction
alpha = 0.05  # Desired FDR level
fdr_results = multipletests(p_values, alpha=alpha, method='fdr_bh')
adjusted_p_values = fdr_results[1]
significant = fdr_results[0]

# Display results
{
    "Original p-values": p_values,
    "Adjusted p-values": adjusted_p_values,
    "Significant predictors": significant
}


{'Original p-values': array([7.93944865e-17, 2.37036494e-08, 1.53522680e-01, 4.52776518e-01,
        9.68885922e-01, 9.17729162e-01, 5.49837859e-01, 2.14392520e-01,
        7.91620325e-01, 6.01574257e-01]),
 'Adjusted p-values': array([7.93944865e-16, 1.18518247e-07, 5.11742265e-01, 8.59391796e-01,
        9.68885922e-01, 9.68885922e-01, 8.59391796e-01, 5.35981300e-01,
        9.68885922e-01, 8.59391796e-01]),
 'Significant predictors': array([ True,  True, False, False, False, False, False, False, False,
        False])}

In [4]:
a = fdr_correction(p_values=[0,0,0,0,0], alpha=0.05)

In [5]:
a[0]

0.0